In [34]:
from glob import glob 
import os 
import time
import geopandas as gpd
from os.path import basename, join
from concurrent.futures import ProcessPoolExecutor

In [35]:
def tiff2vector(tif,figpkg,fogpkg):
    gdal_polygonize(tif, figpkg)
    polygonize_to_cutline(figpkg, fogpkg)

def gdal_polygonize(tif, gpkg):
    if not os.path.isfile(gpkg):
        t1 = time.time()
        cmd_polygonize = f"gdal_polygonize.py -overwrite {tif} -f GPKG {gpkg}"
        os.system(cmd_polygonize)
        t2 = time.time()
        print(f"time cmd_outline:{t2-t1}")
        print(f"Polygon gdal_polygonize saved to {gpkg}")
    else:
        print(f"file areaddy created {gpkg}")
    print('gdal_polygonize')


def polygonize_to_cutline(figpkg, fogpkg):
    if not os.path.isfile(fogpkg):
        t1 = time.time()
        print('reading...')
        gdf = gpd.read_file(figpkg)
        print('dissolving...')
        diss = gdf.dissolve()
        print('writting...')
        diss.to_file(fogpkg, driver="GPKG")
        t2 = time.time()
        print(f"time polygonize_to_cutline:{t2-t1}")
        print(f"Outline file  saved to {fogpkg}")
    else:
        print(f"file areaddy created {fogpkg}")
    print('polygonize_to_cutline')


In [40]:
cutline_dir = "/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/cutline"
tls_fn = "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/AngorW/TLSLidarDEM/AngkorDTM50cm.tif"
mkd_fn = "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/Mekong/delta_s_m2070.tif"
pband_fn = "/media/ljp238/12TBWolf/ARCHIEVE/PBAND_DTM/RNG/NegroAOIDTM.tif"
rng_path = "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_OverlapPatchesLiDAR/"
rng_files = glob(f"{rng_path}/*.tif")
files = rng_files + [tls_fn,mkd_fn, pband_fn]
files = rng_files + [tls_fn, mkd_fn, pband_fn]
os.makedirs(cutline_dir, exist_ok=True)
#tif = files[0]

In [ ]:
with ProcessPoolExecutor() as pex:
    for tif in files:
        figpkg = join(cutline_dir,basename(tif).replace('.tif', '_poly.gpkg'))
        fogpkg = join(cutline_dir,basename(tif).replace('.tif', '_diss.gpkg'))
        pex.submit(tiff2vector, tif,figpkg,fogpkg)
    #tiff2vector(tif,figpkg,fogpkg)

In [30]:
def gdal_polygonize(tif, gpkg):
    if not os.path.isfile(gpkg):
        t1 = time.time()
        cmd_polygonize = f"gdal_polygonize.py -overwrite {tif} -f GPKG {gpkg}"
        os.system(cmd_polygonize)
        t2 = time.time()
        print(f"time cmd_outline:{t2-t1}")
        print(f"Polygon gdal_polygonize saved to {gpkg}")
    else:
        print(f"file areaddy created {gpkg}")
    print('gdal_polygonize')


def polygonize_to_cutline(figpkg, fogpkg):
    if not os.path.isfile(fogpkg):
        t1 = time.time()
        print('reading...')
        gdf = gpd.read_file(figpkg)
        print('dissolving...')
        diss = gdf.dissolve()
        print('writting...')
        diss.to_file(fogpkg, driver="GPKG")
        t2 = time.time()
        print(f"time polygonize_to_cutline:{t2-t1}")
        print(f"Outline file  saved to {fogpkg}")
    else:
        print(f"file areaddy created {fogpkg}")
    print('polygonize_to_cutline')


In [9]:
import os
import time

# Step 1: Define input and output paths
files = rng_files + [tls_fn, mkd_fn, pband_fn]
input_tif = files[0]
output_gpkg = os.path.basename(input_tif).replace('.tif', '_p.gpkg')
merged_gpkg = os.path.basename(input_tif).replace('.tif', 'm_.gpkg')
outline_gpkg = os.path.basename(input_tif).replace('.tif', 'o.gpkg')

# Step 2: Run gdal_polygonize.py to convert raster to vector polygons
t1 = time.time()
cmd_polygonize = f"gdal_polygonize.py -overwrite {input_tif} -f GPKG {output_gpkg}"
os.system(cmd_polygonize)
t2 = time.time()
print(f"Time for gdal_polygonize: {t2 - t1:.2f} seconds")
print(f"Polygonized data saved to {output_gpkg}")

# Step 3: Merge all polygons into a single geometry using ogr2ogr
cmd_merge = f"ogr2ogr -f GPKG {merged_gpkg} {output_gpkg} -dialect SQLITE -sql \"SELECT ST_Union(geometry) AS geometry FROM output\""
os.system(cmd_merge)
t3 = time.time()
print(f"Time for merging polygons: {t3 - t2:.2f} seconds")
print(f"Merged geometry saved to {merged_gpkg}")

# Step 4: Extract the boundary of the merged geometry
cmd_outline = f"ogr2ogr -f GPKG {outline_gpkg} {merged_gpkg} -nlt MULTILINESTRING -sql \"SELECT ST_Boundary(geometry) AS geometry FROM merged\""
os.system(cmd_outline)
t4 = time.time()
print(f"Time for extracting outline: {t4 - t3:.2f} seconds")
print(f"Polygon outline saved to {outline_gpkg}")

Creating output DTM_NP_T-0007_p.gpkg of format GPKG.
0...10...20...30...40...50...60...70...80...90...100 - done.
Time for gdal_polygonize: 5.01 seconds
Polygonized data saved to DTM_NP_T-0007_p.gpkg
Time for merging polygons: 0.07 seconds
Merged geometry saved to DTM_NP_T-0007m_.gpkg
Time for extracting outline: 0.06 seconds
Polygon outline saved to DTM_NP_T-0007o.gpkg


ERROR 1: In ExecuteSQL(): sqlite3_prepare_v2(SELECT ST_Union(geometry) AS geometry FROM output):
  no such table: output
ERROR 1: Unable to open datasource `DTM_NP_T-0007m_.gpkg' with the following drivers.
  -> `PCIDSK'
  -> `PDS4'
  -> `VICAR'
  -> `MBTiles'
  -> `EEDA'
  -> `OGCAPI'
  -> `ESRI Shapefile'
  -> `MapInfo File'
  -> `UK .NTF'
  -> `LVBAG'
  -> `OGR_SDTS'
  -> `S57'
  -> `DGN'
  -> `OGR_VRT'
  -> `Memory'
  -> `CSV'
  -> `NAS'
  -> `GML'
  -> `GPX'
  -> `LIBKML'
  -> `KML'
  -> `GeoJSON'
  -> `GeoJSONSeq'
  -> `ESRIJSON'
  -> `TopoJSON'
  -> `Interlis 1'
  -> `Interlis 2'
  -> `OGR_GMT'
  -> `GPKG'
  -> `SQLite'
  -> `WAsP'
  -> `OpenFileGDB'
  -> `DXF'
  -> `CAD'
  -> `FlatGeobuf'
  -> `Geoconcept'
  -> `GeoRSS'
  -> `VFK'
  -> `PGDUMP'
  -> `OSM'
  -> `GPSBabel'
  -> `OGR_PDS'
  -> `WFS'
  -> `OAPIF'
  -> `EDIGEO'
  -> `SVG'
  -> `Idrisi'
  -> `ODS'
  -> `XLSX'
  -> `Elasticsearch'
  -> `Carto'
  -> `AmigoCloud'
  -> `SXF'
  -> `Selafin'
  -> `JML'
  -> `PLSCENES'
  ->

In [ ]:
import geopandas as gpd
gdf = gpd.read_file(output_gpkg)
dis = gdf.dissolve()
dis.to_file(outline_gpkg, driver="GPKG")

In [20]:
dis

,geometry,DN
0,"POLYGON ((526461 9977822, 526461 9977823, 5264...",28


In [22]:
outline_gpkg

'DTM_NP_T-0007o.gpkg'

In [14]:
type(gdf)

geopandas.geodataframe.GeoDataFrame

In [15]:
# compare the polygonize and the raster values df: shape, values

In [16]:
gdf = gdf.buffer(0)

In [ ]:
gdf.diss

0         POLYGON ((526595 9978356, 526596 9978356, 5265...
1         POLYGON ((526635 9978355, 526637 9978355, 5266...
2         POLYGON ((526591 9978352, 526592 9978352, 5265...
3         POLYGON ((526650 9978351, 526654 9978351, 5266...
4         POLYGON ((526657 9978348, 526660 9978348, 5266...
                                ...                        
128160    POLYGON ((540929 9973495, 540930 9973495, 5409...
128161    POLYGON ((540917 9973484, 540918 9973484, 5409...
128162    POLYGON ((536014 9975544, 536016 9975544, 5360...
128163    POLYGON ((540919 9973486, 540920 9973486, 5409...
128164    POLYGON ((540922 9973481, 540923 9973481, 5409...
Length: 128165, dtype: geometry